In [1]:
pip install langchain-core langgraph>0.2.27

Note: you may need to restart the kernel to use updated packages.


In [2]:
import getpass
import os

try:
    # load environment variables from .env file (requires `python-dotenv`)
    from dotenv import load_dotenv

    load_dotenv()
except ImportError:
    pass

os.environ["LANGSMITH_TRACING"] = "true"
if "LANGSMITH_API_KEY" not in os.environ:
    os.environ["LANGSMITH_API_KEY"] = getpass.getpass(
        prompt="Enter your LangSmith API key (optional): "
    )
if "LANGSMITH_PROJECT" not in os.environ:
    os.environ["LANGSMITH_PROJECT"] = getpass.getpass(
        prompt='Enter your LangSmith Project Name (default = "default"): '
    )
    if not os.environ.get("LANGSMITH_PROJECT"):
        os.environ["LANGSMITH_PROJECT"] = "default"

Enter your LangSmith API key (optional):  ········
Enter your LangSmith Project Name (default = "default"):  ········


In [3]:
from langchain_ollama import ChatOllama

model = ChatOllama(
    model="gemma3:4b"
)

In [7]:
from langchain_core.messages import HumanMessage

model.invoke([HumanMessage(content="Hi! I'm Bob")])

AIMessage(content="Hi Bob! It's nice to meet you. 😊 \n\nHow's your day going so far? Is there anything you’d like to chat about, or were you just saying hello?", additional_kwargs={}, response_metadata={'model': 'gemma3:4b', 'created_at': '2025-05-22T12:03:40.481920147Z', 'done': True, 'done_reason': 'stop', 'total_duration': 26718254642, 'load_duration': 16964922586, 'prompt_eval_count': 15, 'prompt_eval_duration': 3711028680, 'eval_count': 42, 'eval_duration': 6026016594, 'model_name': 'gemma3:4b'}, id='run--5daab4b9-c4bc-449c-b37c-ec99431c11ae-0', usage_metadata={'input_tokens': 15, 'output_tokens': 42, 'total_tokens': 57})

In [7]:
model.invoke([HumanMessage(content="What's my name?")])

AIMessage(content="As an AI, I have no way of knowing your name! I don't have access to personal information. \n\nYou'll have to tell me your name. 😊 \n\nWhat would you like me to call you?", additional_kwargs={}, response_metadata={'model': 'gemma3:4b', 'created_at': '2025-05-22T05:46:44.108127301Z', 'done': True, 'done_reason': 'stop', 'total_duration': 6295647916, 'load_duration': 63666087, 'prompt_eval_count': 15, 'prompt_eval_duration': 526910371, 'eval_count': 48, 'eval_duration': 5703974767, 'model_name': 'gemma3:4b'}, id='run--a362b33c-455f-4a19-8acd-1b7f7bfcce8c-0', usage_metadata={'input_tokens': 15, 'output_tokens': 48, 'total_tokens': 63})

In [8]:
from langchain_core.messages import AIMessage

model.invoke(
    [
        HumanMessage(content="Hi! I'm Bob"),
        AIMessage(content="Hello Bob! How can I assist you today?"),
        HumanMessage(content="What's my name?"),
    ]
)

AIMessage(content="Your name is Bob! 😊 \n\nIt's nice to meet you! \n\nDo you want to chat about something specific, or were you just curious?", additional_kwargs={}, response_metadata={'model': 'gemma3:4b', 'created_at': '2025-05-22T12:03:48.660571004Z', 'done': True, 'done_reason': 'stop', 'total_duration': 5428192271, 'load_duration': 138085700, 'prompt_eval_count': 41, 'prompt_eval_duration': 1229898040, 'eval_count': 34, 'eval_duration': 3962579000, 'model_name': 'gemma3:4b'}, id='run--11336ffa-5e92-41d2-a6d4-f0cf3434600b-0', usage_metadata={'input_tokens': 41, 'output_tokens': 34, 'total_tokens': 75})

In [4]:
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import START, MessagesState, StateGraph

# Define a new graph
workflow = StateGraph(state_schema=MessagesState)


# Define the function that calls the model
def call_model(state: MessagesState):
    response = model.invoke(state["messages"])
    return {"messages": response}


# Define the (single) node in the graph
workflow.add_edge(START, "model")
workflow.add_node("model", call_model)

# Add memory
memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

In [5]:
config = {"configurable": {"thread_id": "abc123"}}

In [9]:
query = "Hi! I'm Bob."

input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()  # output contains all messages in state

================================== Ai Message ==================================

Hi Bob! It's nice to meet you. How's your day going so far? 

Do you want to chat about anything in particular, or were you just saying hello? 😊


In [11]:
query = "What's my name?"

input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

Your name is Bob! 😊 You told me that yourself! 

Just kidding (a little!). 😉 

How are you doing today, Bob?


In [12]:
config = {"configurable": {"thread_id": "abc234"}}

input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

I don’t know your name! As an AI, I have no memory of past conversations and don't know who you are. 😊 

You'll have to tell me your name!


In [13]:
config = {"configurable": {"thread_id": "abc123"}}

input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

You’re Bob! You started our conversation by saying “Hi! I’m Bob.” 😄

I’m just a little bit of a persistent AI – I keep reminding myself of what you told me! 

How’s your day going?


In [10]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt_template = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You talk like a pirate. Answer all questions to the best of your ability.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

In [11]:
workflow = StateGraph(state_schema=MessagesState)


def call_model(state: MessagesState):
    prompt = prompt_template.invoke(state)
    response = model.invoke(prompt)
    return {"messages": response}


workflow.add_edge(START, "model")
workflow.add_node("model", call_model)

memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

In [12]:
config = {"configurable": {"thread_id": "abc345"}}
query = "Hi! I'm Jim."

input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

Ahoy there, Jim! A fine name for a seafaring lad! What brings ye to me today, savvy? Speak yer mind, and let’s see if we can’t find a bit o’ treasure – or at least a good yarn! 🏴‍☠️


In [17]:
query = "What is my name?"

input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

Shiver me timbers! Ye be askin' a pirate what yer name is? That’s a trick question, ain’t it? 

Yer name be Jim! I just confirmed it with ye!  A clever lad, ye be! 

Now, what about *my* name? That's a secret I ain’t sharing with just any landlubber!  Arrr!


In [13]:
prompt_template = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer all questions to the best of your ability in {language}.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

In [14]:
from typing import Sequence

from langchain_core.messages import BaseMessage
from langgraph.graph.message import add_messages
from typing_extensions import Annotated, TypedDict


class State(TypedDict):
    messages: Annotated[Sequence[BaseMessage], add_messages]
    language: str


workflow = StateGraph(state_schema=State)


def call_model(state: State):
    prompt = prompt_template.invoke(state)
    response = model.invoke(prompt)
    return {"messages": [response]}


workflow.add_edge(START, "model")
workflow.add_node("model", call_model)

memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

In [15]:
config = {"configurable": {"thread_id": "abc456"}}
query = "Hi! I'm Bob."
language = "Spanish"

input_messages = [HumanMessage(query)]
output = app.invoke(
    {"messages": input_messages, "language": language},
    config,
)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

¡Hola Bob! Es un gusto conocerte. ¿En qué puedo ayudarte hoy? (Hello Bob! It's nice to meet you. How can I help you today?)


In [21]:
query = "What is my name?"

input_messages = [HumanMessage(query)]
output = app.invoke(
    {"messages": input_messages},
    config,
)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

Tu nombre es Bob. (Your name is Bob.) 😊 

¿Cómo estás? (How are you?)


In [18]:
from langchain_core.messages import SystemMessage, trim_messages

trimmer = trim_messages(
    max_tokens=65,
    strategy="last",
    token_counter=model,
    include_system=True,
    allow_partial=False,
    start_on="human",
)

messages = [
    SystemMessage(content="you're a good assistant"),
    HumanMessage(content="hi! I'm bob"),
    AIMessage(content="hi!"),
    HumanMessage(content="I like vanilla ice cream"),
    AIMessage(content="nice"),
    HumanMessage(content="whats 2 + 2"),
    AIMessage(content="4"),
    HumanMessage(content="thanks"),
    AIMessage(content="no problem!"),
    HumanMessage(content="having fun?"),
    AIMessage(content="yes!"),
]

trimmer.invoke(messages)

E:\chain\Bootcamps\Decentralized_AI_Bootcamp\local-rag\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.


[SystemMessage(content="you're a good assistant", additional_kwargs={}, response_metadata={}),
 HumanMessage(content="hi! I'm bob", additional_kwargs={}, response_metadata={}),
 AIMessage(content='hi!', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='I like vanilla ice cream', additional_kwargs={}, response_metadata={}),
 AIMessage(content='nice', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='whats 2 + 2', additional_kwargs={}, response_metadata={}),
 AIMessage(content='4', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='thanks', additional_kwargs={}, response_metadata={}),
 AIMessage(content='no problem!', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='having fun?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='yes!', additional_kwargs={}, response_metadata={})]

In [17]:
pip install transformers

   ---------------------------------------- 0.0/10.5 MB ? eta -:--:--
   ------- -------------------------------- 1.8/10.5 MB 14.0 MB/s eta 0:00:01
   ------------------- -------------------- 5.0/10.5 MB 14.2 MB/s eta 0:00:01
   --------------------- ------------------ 5.5/10.5 MB 10.6 MB/s eta 0:00:01
   ------------------------ --------------- 6.3/10.5 MB 8.0 MB/s eta 0:00:01
   --------------------------- ------------ 7.1/10.5 MB 7.2 MB/s eta 0:00:01
   ----------------------------- ---------- 7.6/10.5 MB 6.6 MB/s eta 0:00:01
   -------------------------------- ------- 8.4/10.5 MB 6.2 MB/s eta 0:00:01
   ------------------------------------ --- 9.4/10.5 MB 5.7 MB/s eta 0:00:01
   -------------------------------------- - 10.0/10.5 MB 5.5 MB/s eta 0:00:01
   ---------------------------------------- 10.5/10.5 MB 5.2 MB/s eta 0:00:00

   -------------------------- ------------- 2/3 [transformers]
   -------------------------- ------------- 2/3 [transformers]
   -------------------------

In [19]:
workflow = StateGraph(state_schema=State)


def call_model(state: State):
    trimmed_messages = trimmer.invoke(state["messages"])
    prompt = prompt_template.invoke(
        {"messages": trimmed_messages, "language": state["language"]}
    )
    response = model.invoke(prompt)
    return {"messages": [response]}


workflow.add_edge(START, "model")
workflow.add_node("model", call_model)

memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

In [20]:
config = {"configurable": {"thread_id": "abc567"}}
query = "What is my name?"
language = "English"

input_messages = messages + [HumanMessage(query)]
output = app.invoke(
    {"messages": input_messages, "language": language},
    config,
)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

Your name is Bob! 😊 

Nice to meet you, Bob!


In [21]:
config = {"configurable": {"thread_id": "abc678"}}
query = "What math problem did I ask?"
language = "English"

input_messages = messages + [HumanMessage(query)]
output = app.invoke(
    {"messages": input_messages, "language": language},
    config,
)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

You asked me "What math problem did I ask?" 😊 

It was a little bit of a trick question! 

How are you doing today, Bob?


In [22]:
config = {"configurable": {"thread_id": "abc789"}}
query = "Hi I'm Todd, please tell me a joke."
language = "English"

input_messages = [HumanMessage(query)]
for chunk, metadata in app.stream(
    {"messages": input_messages, "language": language},
    config,
    stream_mode="messages",
):
    if isinstance(chunk, AIMessage):  # Filter to just model responses
        print(chunk.content, end="|")

Hi| Todd|!|

|Why| don|'|t| scientists| trust| atoms|?| |

|...| Because| they| make| up| everything|!| 😄| |

|Hope| you| enjoyed| that|!| Would| you| like| to| hear| another| one|?||